# Preparación de los datos para el sistema de recomendación

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from joblib import dump, load
from sklearn.preprocessing import normalize

In [3]:
# Cargo cv con la informacion de cada compra. Aquí se deben de incluir todo tipo de clientes
df_compras = pd.read_csv('../data/clean_dataset_lat_long.csv') 
df_compras.head()

,FECHA,CLIENTE,DESCRIPCION,UNIDADES,ARTICULO,CODIGO_COMPRA,FRECUENCIA_ANUAL,VENTA_ANUAL,codigo_familia,codigo_fabricante,Poblacion,codigo_grupo,precio_unitario_final,antiguedad,Latitude,Longitude
0,2024-02-29 00:00:00.000,1019,GRIFO LLENAVASOS PARA FUENTES DE AGUA REF. 100...,1.0,002810071,SM1-3128,29,28077.52,0028,40007404,GANDIA,1.0,26.460,7239,38.967593,-0.180342
1,2024-07-31 00:00:00.000,1019,TUBO DE ENLACE KOMBIFIX CON TUERCAS REF. 152.4...,1.0,0044152426461,SM1-11147,29,28077.52,0044,40000662,GANDIA,1.0,23.530,7239,38.967593,-0.180342
2,2021-01-29 00:00:00.000,1019,FLOTADOR UNIFILL REF. 240.705.00.1 PARA CISTER...,1.0,0044240705001,SM1-1785,21,6723.96,0044,40000662,GANDIA,1.0,24.375,7239,38.967593,-0.180342
3,2024-04-30 00:00:00.000,1019,Plato de ducha de carga mineral textura pizarr...,1.0,0060PDSGPZ10070BL,SM1-6071,29,28077.52,0064,40000772,GANDIA,1.0,177.750,7239,38.967593,-0.180342
4,2023-05-31 00:00:00.000,1019,VERTEDERO MODELO GARDA A371055000 ROCA,1.0,00R00601,SM1-8041,51,24813.50,00R0,40000138,GANDIA,1.0,141.000,7239,38.967593,-0.180342


In [4]:
# Eliminar duplicados para obtener un DataFrame limpio con ARTICULO y DESCRIPCION
productos = df_compras[['ARTICULO', 'DESCRIPCION']].drop_duplicates().reset_index(drop=True)
productos

,ARTICULO,DESCRIPCION
0,002810071,GRIFO LLENAVASOS PARA FUENTES DE AGUA REF. 100...
1,0044152426461,TUBO DE ENLACE KOMBIFIX CON TUERCAS REF. 152.4...
2,0044240705001,FLOTADOR UNIFILL REF. 240.705.00.1 PARA CISTER...
3,0060PDSGPZ10070BL,Plato de ducha de carga mineral textura pizarr...
4,00R00601,VERTEDERO MODELO GARDA A371055000 ROCA
...,...,...
19846,0901304498,Manguito portabrida 315 pvc presión
19847,0901304500,Brida loca presión 315 de 12 agujeros
19848,090658707,"SKIMMER 17,5L NORM para piscina de hormigón b..."
19849,1012121NE,Fregadero sobre encimera CORAL de 1 cubeta y e...


In [5]:
# Guardamos productos en csv
productos.to_csv('../data/productos.csv')

In [6]:
# Preparo un df a partir del df_compras en el que voy a agrupar los artículos por transacción.
# Agrupar por 'CODIGO_COMPRA'
compras = df_compras.groupby('CODIGO_COMPRA').agg(Articulos=('ARTICULO', lambda x: list(x)), Unidades=('UNIDADES', lambda x: list(x))).reset_index()

# Mostrar el nuevo DataFrame
compras.head()

,CODIGO_COMPRA,Articulos,Unidades
0,0R8-6,[0804133287],[15.0]
1,1-1,"[014697462000, 0438UAES]","[1.0, 1.0]"
2,1-10,"[0274IGTH221222, 077073154N, 2001WPDIAZ]","[4.0, 1.0, 1.0]"
3,1-11,"[0274AI70M42CAD, 0274AI70M54CAD, 0033TO0051B]","[1.0, 1.0, 1.0]"
4,1-12,"[07R2A851693509, 0060PDSGPZ13080BL, 2003WPMAHBL]","[1.0, 1.0, 1.0]"


In [7]:
compras.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61396 entries, 0 to 61395
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   CODIGO_COMPRA  61396 non-null  object
 1   Articulos      61396 non-null  object
 2   Unidades       61396 non-null  object
dtypes: object(3)
memory usage: 1.4+ MB


# Cogemos las compras que tengan mas de un artículo

In [18]:
# Filtrar las cestas que tienen más de un artículo
compras_filtradas = compras[compras['Articulos'].apply(len) > 1]

# Mostrar el DataFrame filtrado
compras_filtradas.head()

,CODIGO_COMPRA,Articulos,Unidades
1,1-1,"[014697462000, 0438UAES]","[1.0, 1.0]"
2,1-10,"[0274IGTH221222, 077073154N, 2001WPDIAZ]","[4.0, 1.0, 1.0]"
3,1-11,"[0274AI70M42CAD, 0274AI70M54CAD, 0033TO0051B]","[1.0, 1.0, 1.0]"
4,1-12,"[07R2A851693509, 0060PDSGPZ13080BL, 2003WPMAHBL]","[1.0, 1.0, 1.0]"
7,1-15,"[0274AI20AM042040, 0274IGCV45HM42]","[12.0, 2.0]"


# Archivo para probar sin tener en cuenta las unidades: cestas_su.csv


In [19]:
cestas_su = pd.DataFrame({ 'Cestas': compras_filtradas.apply(lambda row: ' '.join([item for item in row['Articulos']]), axis=1)})

In [20]:
cestas_su.head()

,Cestas
1,014697462000 0438UAES
2,0274IGTH221222 077073154N 2001WPDIAZ
3,0274AI70M42CAD 0274AI70M54CAD 0033TO0051B
4,07R2A851693509 0060PDSGPZ13080BL 2003WPMAHBL
7,0274AI20AM042040 0274IGCV45HM42


In [21]:
cestas_su.info

<bound method DataFrame.info of                                                   Cestas
1                                  014697462000 0438UAES
2                   0274IGTH221222 077073154N 2001WPDIAZ
3              0274AI70M42CAD 0274AI70M54CAD 0033TO0051B
4           07R2A851693509 0060PDSGPZ13080BL 2003WPMAHBL
7                        0274AI20AM042040 0274IGCV45HM42
...                                                  ...
61387  0201CU2431210 0201CU2431512 0201CU2715 0422VI1...
61388  00R9822502100 00R9822502400 00R9V0028800R 0226...
61393                                  1600ES12 1600ES34
61394              0803B2003000 0804241250200 0816270250
61395                                  0660046 069551010

[59367 rows x 1 columns]>

# Prueba teniendo en cuenta unidades: cestas_csv


In [11]:
# Convertir los artículos en una sola cadena, multiplicando por la cantidad de unidades. Lo pongo en la columna cestas
cestas = pd.DataFrame({ 'Cestas': compras.apply(lambda row: ' '.join([f"{item} " * int(unit) for item, unit in zip(row['Articulos'], row['Unidades'])]), axis=1)})

In [12]:
cestas.head()

,Cestas
0,0804133287 0804133287 0804133287 0804133287 08...
1,014697462000 0438UAES
2,0274IGTH221222 0274IGTH221222 0274IGTH221222 0...
3,0274AI70M42CAD 0274AI70M54CAD 0033TO0051B
4,07R2A851693509 0060PDSGPZ13080BL 2003WPMAHBL


In [13]:
cestas.info

<bound method DataFrame.info of                                                   Cestas
0      0804133287 0804133287 0804133287 0804133287 08...
1                                014697462000  0438UAES 
2      0274IGTH221222 0274IGTH221222 0274IGTH221222 0...
3           0274AI70M42CAD  0274AI70M54CAD  0033TO0051B 
4        07R2A851693509  0060PDSGPZ13080BL  2003WPMAHBL 
...                                                  ...
61391                               200650627 200650627 
61392                                     0780ME4002021 
61393  1600ES12 1600ES12 1600ES12 1600ES12 1600ES12 1...
61394           0803B2003000  0804241250200  0816270250 
61395                                0660046  069551010 

[61396 rows x 1 columns]>

In [14]:
# Guardamos productos en csv
cestas.to_csv('../data/cestas.csv')

In [22]:
# Guardamos productos en csv
cestas_su.to_csv('../data/cestas_su.csv')

# Para probar el método de vectorización usamos las cestas con unidades

Vectorización de cestas usando TFIDF

In [49]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(cestas['Cestas'])

In [50]:
# Guardar la matriz TF-IDF y el modelo
dump(tfidf_matrix, '../models/tfidf_matrix.joblib') # matriz
dump(tfidf, '../models/tfidf_model.joblib') # modelo de vectorizacion

['../models/tfidf_model.joblib']

Vectorización de cestas usando COuntVEctorizer

In [69]:
all_cestas = cestas['Cestas'].tolist() + cestas_su['Cestas'].tolist()

In [70]:
count_vectorizer = CountVectorizer()
count_vectorizer.fit(all_cestas)

CountVectorizer()

In [71]:
# Calcular la matriz de conteo
count_matrix = count_vectorizer.transform(cestas['Cestas'])

In [72]:
# Guardar la matriz de conteo y el modelo en la carpeta 'models'
dump(count_matrix, '../models/count_matrix.joblib')
dump(count_vectorizer, '../models/count_vectorizer.joblib')

['../models/count_vectorizer.joblib']

Vectorizacion usando count + normalizacion L1

In [73]:
count_matrix = count_vectorizer.transform(cestas['Cestas'])
# Normalizar la matriz de conteo por la suma total de términos en cada documento (aplicar TF)
tf_matrix = normalize(count_matrix, norm='l1')  # 'l1' normaliza por la suma de la fila

In [74]:
tf_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 689038 stored elements and shape (61396, 18912)>

In [75]:
# Guardar la matriz de conteo y el modelo en la carpeta 'models'
dump(tf_matrix, '../models/tf_matrix.joblib')

['../models/tf_matrix.joblib']

# Para probar sin unidades cojo directamente count

In [76]:
count_matrix_su = count_vectorizer.transform(cestas_su['Cestas'])
# Normalizar la matriz de conteo por la suma total de términos en cada documento (aplicar TF)
tf_matrix_su = normalize(count_matrix_su, norm='l1')  # 'l1' normaliza por la suma de la fila

In [77]:
tf_matrix_su

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 689067 stored elements and shape (61396, 18912)>

In [78]:
# Guardar la matriz de conteo y el modelo en la carpeta 'models'
dump(tf_matrix_su, '../models/tf_matrix_su.joblib')

['../models/tf_matrix_su.joblib']